In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import html
import collections
from collections import Counter

In [ ]:
#Expectes data to be in the same directory as the notebook in a folder called 'archive'
path_book_ratings = "archive/Books_rating.csv"
br = pd.read_csv(path_book_ratings)

## Filter columns


In [3]:
br = br[['Id','review/score', 'review/text']]

In [6]:
br.shape

(2999992, 3)

## Drop nas


In [5]:
br = br.dropna(subset=['review/text', 'review/score'])

## Replace HTML entities

In [7]:
br['cleanText'] = br['review/text'].astype(str).apply(html.unescape)

## Remove URLs, control characters, tabs, newlines and replace multiple Spaces by one

In [10]:
# --- Combined cleaning function ---
def clean_text_all(text):
    text = str(text)

    # 1. Remove control characters (non-printable ASCII)
    # \x00-\x1F are control chars, \x7F is DEL
    text = re.sub(r'[\x00-\x1F\x7F]', '', text)

    # 2. Remove URLs (http/https/www)
    text = re.sub(r'http\S+|www\S+', '', text)

    # 3. Replace newlines, carriage returns, tabs with a single space
    text = re.sub(r'[\r\n\t]+', ' ', text)

    # 4. Normalize multiple spaces to one and strip leading/trailing spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# --- Apply to your dataframe ---
br['cleanText'] = br['cleanText'].apply(clean_text_all)

print("✅ Cleaning complete.")


✅ Cleaning complete.


## Check
Check if there are any URL, HTML entities, extra whitespaces or control characters left 
And check if any of these is still in there: ['\x01', '\x11', '\x7f', '\x17', '\x13', '\x14', '\x16', '\x04', '\x18', '\x1d', '\x0f', '\x1b', '\x1f', '\x05', '\x10', '\x19', '\x0e', '\x08', '\x06', '\x02', '\x07', '\x15']

In [12]:
mask_ctrl = br['cleanText'].str.contains(r'[\x00-\x1F\x7F]', regex=True)
print(f"Rows still containing control characters: {mask_ctrl.sum()}")


Rows still containing control characters: 0


In [13]:
mask_url = br['cleanText'].str.contains(r'http\S+|www\S+', regex=True)
print(f"Rows still containing URLs: {mask_url.sum()}")


Rows still containing URLs: 0


In [14]:
mask_space = br['cleanText'].str.contains(r'\s{2,}', regex=True)
print(f"Rows still containing double/multiple spaces: {mask_space.sum()}")


Rows still containing double/multiple spaces: 0


In [15]:
control_chars_list = ['\x01', '\x11', '\x7f', '\x17', '\x13', '\x14', '\x16',
                      '\x04', '\x18', '\x1d', '\x0f', '\x1b', '\x1f', '\x05',
                      '\x10', '\x19', '\x0e', '\x08', '\x06', '\x02', '\x07', '\x15']

pattern = '|'.join(re.escape(ch) for ch in control_chars_list)


# --- Check if any of these characters remain ---
mask = br['cleanText'].astype(str).str.contains(pattern, regex=True)
count_remaining = mask.sum()
print(f"Rows still containing specified control characters: {count_remaining}")

Rows still containing specified control characters: 0


## Safe cleaned up data in a new file


In [ ]:
#safe dataframe columns 'Id', 'review/score', 'cleanText' to a new csv file
br[['Id', 'review/score', 'cleanText']].to_csv('archive/Books_rating_cleaned.csv', index=False)

OSError: [Errno 28] No space left on device